<a href="https://colab.research.google.com/github/akshay4n/Quantitaive-Research/blob/main/quantitative_momentum_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 7.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [3]:
path = '/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500/sp_500_stocks.csv'
stocks = pd.read_csv(path)

In [4]:
%cd /content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500/
from secrets_imp import IEX_CLOUD_API_TOKEN

/content/drive/MyDrive/QuantitativeResearch/EqualWeight_SP500


In [5]:
#one year price returns for each stock
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# data

data['year1ChangePercent']

0.19898482282705007

In [6]:
#batch api calls
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

symbol_groups = list(chunks(stocks['Ticker'][1:116],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker','Price', 'One-Year Price Return','Number of Shares to Buy']

In [7]:
final_df = pd.DataFrame(columns = my_columns)
# final_df

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )

final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAL,15.13,-0.214901,N/A
1,AAP,208.11,0.021943,N/A
2,AAPL,176.21,0.203401,N/A
3,ABBV,145.52,0.249686,N/A
4,ABC,157.20,0.275778,N/A
...,...,...,...,...
110,COTY,8.00,0.007755,N/A
111,CPB,52.18,0.222787,N/A
112,CPRT,134.64,-0.053614,N/A
113,CRM,188.20,-0.265988,N/A


In [8]:
#remove low-momentum stocks
final_df.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,88,CF,106.07,1.435917,N/A
1,41,APA,37.80,1.374344,N/A
2,108,COP,108.23,1.089868,N/A
3,10,ADM,90.32,0.507803,N/A
4,103,CNC,97.48,0.497927,N/A
5,36,ANET,137.24,0.466368,N/A
6,93,CI,302.36,0.445115,N/A
7,18,AIV,9.58,0.442764,N/A
8,53,AZO,2419.71,0.423299,N/A
9,76,CAH,71.10,0.402682,N/A


In [9]:
#calculate number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')

portfolio_input()

Enter the size of your portfolio100000000


In [10]:
position_size = float(portfolio_size)/len(final_df)
for i in range(0,len(final_df)):
    final_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i,'Price'])

final_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,88,CF,106.07,1.435917,18855
1,41,APA,37.80,1.374344,52910
2,108,COP,108.23,1.089868,18479
3,10,ADM,90.32,0.507803,22143
4,103,CNC,97.48,0.497927,20517
5,36,ANET,137.24,0.466368,14573
6,93,CI,302.36,0.445115,6614
7,18,AIV,9.58,0.442764,208768
8,53,AZO,2419.71,0.423299,826
9,76,CAH,71.10,0.402682,28129


In [11]:
#high quality momentum stock(hqm) and low quality momentum stock(lqm)
hqm_column = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns = hqm_column)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_column
            ),
            ignore_index=True
        )
# hqm_df['One-Year Price Return'].fillna(0)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAL,15.40,N/A,-0.209580,N/A,-0.173540,N/A,-0.114684,N/A,-0.021226,N/A,N/A
1,AAP,213.82,N/A,0.021702,N/A,-0.036900,N/A,0.069968,N/A,0.092480,N/A,N/A
2,AAPL,177.46,N/A,0.203191,N/A,0.044434,N/A,0.282943,N/A,0.159858,N/A,N/A
3,ABBV,146.09,N/A,0.253924,N/A,-0.001498,N/A,-0.060389,N/A,-0.058264,N/A,N/A
4,ABC,151.94,N/A,0.279396,N/A,0.066809,N/A,0.006956,N/A,0.052213,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
110,COTY,8.19,N/A,0.007984,N/A,-0.152026,N/A,0.227901,N/A,0.024658,N/A,N/A
111,CPB,51.84,N/A,0.219780,N/A,0.153881,N/A,0.113526,N/A,0.051378,N/A,N/A
112,CPRT,133.72,N/A,-0.052832,N/A,0.066649,N/A,0.178645,N/A,0.074597,N/A,N/A
113,CRM,193.95,N/A,-0.266169,N/A,-0.046685,N/A,0.214839,N/A,0.082441,N/A,N/A


In [12]:
#calculating momentum percentiles
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_df.index:
    for time_period in time_periods:
        # hqm_df.loc[row,f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'],hqm_df.loc[row, f'{time_period} Price Return'])

        hqm_df.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_df[f'{time_period} Price Return'], hqm_df.loc[row, f'{time_period} Price Return'])
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAL,15.40,N/A,-0.209580,16.521739,-0.173540,14.782609,-0.114684,6.956522,-0.021226,9.565217,N/A
1,AAP,213.82,N/A,0.021702,53.913043,-0.036900,39.130435,0.069968,47.826087,0.092480,53.043478,N/A
2,AAPL,177.46,N/A,0.203191,77.391304,0.044434,59.130435,0.282943,93.043478,0.159858,78.26087,N/A
3,ABBV,146.09,N/A,0.253924,82.608696,-0.001498,46.956522,-0.060389,9.565217,-0.058264,5.217391,N/A
4,ABC,151.94,N/A,0.279396,86.086957,0.066809,66.086957,0.006956,25.217391,0.052213,31.304348,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
110,COTY,8.19,N/A,0.007984,51.304348,-0.152026,19.130435,0.227901,85.217391,0.024658,22.608696,N/A
111,CPB,51.84,N/A,0.219780,79.130435,0.153881,81.73913,0.113526,58.26087,0.051378,30.434783,N/A
112,CPRT,133.72,N/A,-0.052832,38.26087,0.066649,65.217391,0.178645,75.652174,0.074597,39.130435,N/A
113,CRM,193.95,N/A,-0.266169,10.434783,-0.046685,36.521739,0.214839,84.347826,0.082441,44.347826,N/A


In [13]:
#calculating HQM score
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row,'HQM Score'] = mean(momentum_percentiles)

hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAL,15.40,N/A,-0.209580,16.521739,-0.173540,14.782609,-0.114684,6.956522,-0.021226,9.565217,11.956522
1,AAP,213.82,N/A,0.021702,53.913043,-0.036900,39.130435,0.069968,47.826087,0.092480,53.043478,48.478261
2,AAPL,177.46,N/A,0.203191,77.391304,0.044434,59.130435,0.282943,93.043478,0.159858,78.26087,76.956522
3,ABBV,146.09,N/A,0.253924,82.608696,-0.001498,46.956522,-0.060389,9.565217,-0.058264,5.217391,36.086957
4,ABC,151.94,N/A,0.279396,86.086957,0.066809,66.086957,0.006956,25.217391,0.052213,31.304348,52.173913
...,...,...,...,...,...,...,...,...,...,...,...,...
110,COTY,8.19,N/A,0.007984,51.304348,-0.152026,19.130435,0.227901,85.217391,0.024658,22.608696,44.565217
111,CPB,51.84,N/A,0.219780,79.130435,0.153881,81.73913,0.113526,58.26087,0.051378,30.434783,62.391304
112,CPRT,133.72,N/A,-0.052832,38.26087,0.066649,65.217391,0.178645,75.652174,0.074597,39.130435,54.565217
113,CRM,193.95,N/A,-0.266169,10.434783,-0.046685,36.521739,0.214839,84.347826,0.082441,44.347826,43.913043


In [14]:
hqm_df.sort_values('HQM Score',ascending=False,inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace=True, drop=True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AIV,9.39,N/A,0.446455,94.782609,0.344212,97.391304,0.596290,100.0,0.285074,98.26087,97.608696
1,CAH,69.60,N/A,0.407101,92.173913,0.303601,94.782609,0.250665,87.826087,0.237315,93.913043,92.173913
2,ALB,288.77,N/A,0.328934,88.695652,0.472319,100.0,0.178593,74.782609,0.325277,100.0,90.869565
3,ADP,264.79,N/A,0.270940,83.478261,0.329798,96.521739,0.280797,92.173913,0.200357,90.434783,90.652174
4,COG,23.06,N/A,0.347578,89.565217,0.188697,86.086957,0.266805,90.434783,0.259281,95.652174,90.434783
5,CDNS,198.79,N/A,0.248041,80.869565,0.427340,99.130435,0.336253,96.521739,0.180555,83.478261,90.0
6,AES,26.29,N/A,0.136279,72.173913,0.253168,91.304348,0.267978,91.304348,0.278234,97.391304,88.043478
7,ANET,133.16,N/A,0.465859,95.652174,0.056479,62.608696,0.320494,94.782609,0.276163,96.521739,87.391304
8,CHRW,123.51,N/A,0.397574,91.304348,0.313868,95.652174,0.180989,76.521739,0.183325,85.217391,87.173913
9,AJG,197.12,N/A,0.368437,90.434783,0.256292,92.173913,0.245016,86.956522,0.152215,76.521739,86.521739


In [15]:
#Number of shares to Buy
portfolio_input()

Enter the size of your portfolio100000000


In [16]:
position_size = float(portfolio_size)/len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i,'Price'])
hqm_df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AIV,9.39,212992,0.446455,94.782609,0.344212,97.391304,0.596290,100.0,0.285074,98.26087,97.608696
1,CAH,69.60,28735,0.407101,92.173913,0.303601,94.782609,0.250665,87.826087,0.237315,93.913043,92.173913
2,ALB,288.77,6925,0.328934,88.695652,0.472319,100.0,0.178593,74.782609,0.325277,100.0,90.869565
3,ADP,264.79,7553,0.270940,83.478261,0.329798,96.521739,0.280797,92.173913,0.200357,90.434783,90.652174
4,COG,23.06,86730,0.347578,89.565217,0.188697,86.086957,0.266805,90.434783,0.259281,95.652174,90.434783
5,CDNS,198.79,10060,0.248041,80.869565,0.427340,99.130435,0.336253,96.521739,0.180555,83.478261,90.0
6,AES,26.29,76074,0.136279,72.173913,0.253168,91.304348,0.267978,91.304348,0.278234,97.391304,88.043478
7,ANET,133.16,15019,0.465859,95.652174,0.056479,62.608696,0.320494,94.782609,0.276163,96.521739,87.391304
8,CHRW,123.51,16193,0.397574,91.304348,0.313868,95.652174,0.180989,76.521739,0.183325,85.217391,87.173913
9,AJG,197.12,10146,0.368437,90.434783,0.256292,92.173913,0.245016,86.956522,0.152215,76.521739,86.521739


In [17]:
#write to excel file
writer = pd.ExcelWriter('momentum_strategy.xlsx',engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name='Momentum Strategy')